In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data/cosmetics.csv')

In [3]:
data.head()

,Unnamed: 0,category,url,brand,name,price,rating,skin_type,ingredients
0,0,beauty-complexion,https://en.zalando.de/darphin-melaperfect-foun...,Darphin,MELAPERFECT FOUNDATION NEUTRAL - Foundation,39.95,4.000000,NaN,"Water\aqua\eau, methyl, trimethicone, phenyl t..."
1,1,beauty-complexion,https://en.zalando.de/nyx-professional-makeup-...,Nyx Professional Makeup,HD PHOTOGENIC CONCEALER WAND - Concealer,6.95,4.143646,NaN,"AQUA / WATER, TRIMETHYLSILOXYPHENYL DIMETHICON..."
2,2,beauty-complexion,https://en.zalando.de/nyx-professional-makeup-...,Nyx Professional Makeup,BARE WITH ME TINTED SKIN VEIL - Setting spray ...,9.95,4.560000,NaN,"Aqua/Water/Eau, Glycerin, Alcohol Denat., Aloe..."
3,3,beauty-complexion,https://en.zalando.de/mac-prep--prime-fix--mat...,MAC,PREP + PRIME FIX + MATTIFYING MIST 100ML - Primer,22.50,4.327586,NaN,"Water\Aqua\Eau , Alcohol Denat. , Silica , Sod..."
4,4,beauty-complexion,https://en.zalando.de/mac-studio-fix-fluid-spf...,MAC,STUDIO FIX FLUID SPF15 FOUNDATION - Foundation,33.95,4.485348,NaN,"Octinoxate 2.50%, Titanium Dioxide 1.00%Water\..."


In [4]:
data.shape

(5392, 9)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5392 entries, 0 to 5391
Data columns (total 9 columns):
Unnamed: 0     5392 non-null int64
category       5392 non-null object
url            5392 non-null object
brand          5029 non-null object
name           5029 non-null object
price          5029 non-null float64
rating         4212 non-null float64
skin_type      1886 non-null object
ingredients    4950 non-null object
dtypes: float64(2), int64(1), object(6)
memory usage: 379.2+ KB


In [13]:
data.isnull().sum()/data.shape[0]*100

category        0.000000
url             0.000000
brand           6.732196
name            6.732196
price           6.732196
rating         21.884273
skin_type      65.022255
ingredients     8.197329
dtype: float64

In [21]:
data.columns

Index(['category', 'url', 'brand', 'name', 'price', 'rating', 'skin_type',
       'ingredients'],
      dtype='object')

нужно удалить пустые значения:
- `Unnamed` не нужен
- `brand` и `name` и `price` около 300 (6.7%) значений без названия, не будем это хранить 

In [6]:
data = data.drop('Unnamed: 0', axis=1)

#### category

In [22]:
data.category.value_counts()

beauty-complexion        1888
beauty-skincare-face     1616
beauty-sun-protection    1525
Name: category, dtype: int64

категории надо сделать по проще

#### url 

In [27]:
data = data.drop(['url'], axis=1)

#### name, brandname, price

In [7]:
data.name.isnull().sum()/data.shape[0]*100

6.732195845697329

In [14]:
data.dropna(subset=['name'], inplace=True)

In [15]:
data.reset_index(drop=True, inplace=True)

In [28]:
data.tail()

,category,brand,name,price,rating,skin_type,ingredients
5024,beauty-sun-protection,Korres,WALNUT & COCONUT FACE & BODY OIL SPF10 150ML -...,19.95,3.000000,NaN,"C12-15 Alkyl Benzoate, Sesamum Indicum (Sesame..."
5025,beauty-sun-protection,Vita Liberata,INVISI MOUSSE 200ML - Self tan,29.95,4.750000,All skin types,"Aloe Barbadensis Leaf Juice *, Dihydroxyaceton..."
5026,beauty-sun-protection,Clinique,SPF30 VIRTU-OIL BODY MIST 144ML - Sun protection,28.95,5.000000,All skin types,"Avobenzone 3.0% , Homosalate 10.0% , Octisalat..."
5027,beauty-sun-protection,St. Tropez,GRADUAL TAN SCULPT & GLOW LOTION 200ML - Self tan,25.95,3.666667,NaN,"Aqua (Water), Cyclopentasiloxane, Dimethicone,..."
5028,beauty-sun-protection,St. Tropez,GRADUAL TAN LUMINOUS VEIL FACE 50ML - Self tan,25.95,2.500000,NaN,"Aqua (Water/Eau), Glycerin, C12-15 Alkyl Benzo..."


#### ingredients

In [19]:
# not too much data with empty ingredients
data.ingredients.isnull().sum()/data.shape[0]*100

1.5708888447007359

In [ ]:
# any duplicates left?
# df_2 = data['name'].drop_duplicates()
# cosm = cosm.loc[df_2.index, :].reset_index()

#### Content-based Recommendation Filtering using Cosine similarities 

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
# from rake_nltk import Rake